In [ ]:
import os
import sys
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

In [ ]:
warnings.filterwarnings('ignore')
path = os.getenv("ROOT_PATH")

> After analysing different periods of time by checking different proxies, we reached some conclusions:
>
> - **Before Covid:** We find significant price reversals when the **price decreases**.
> - **During and after Covid:** We find statistically significant price reversals when the **price increases**.
> 
> Therefore, we are going to create regressions for:
> - Events before covid where the large price change was negative, and their corresponding temporal windows.
> - Events during and after covid where the large price change was positive, and their corresponding temporal windows.
> In order to control the effect of different variables in the abnormal stock returns.


> Variables that we need:
> - ARit: Abnormal stock return from the event i until the end of the post-event window t.
> - MR0_dumi: Dummy variable that takes the value 1  if the market return corresponding to day 0 for event i is positive/negative, and 0 otherwise
> - MCapi: Algorithm of the firm’s market capitalization on the day of the event i.
> - betai: CAPM beta calculated between day -250 and -1 of the event i.
> - SR_volati: Standard deviation of the stock returns calculated between day -250 and -1 of the event i.
> - |SR0|i : Absolut stock return during day 0 of event i.
> - ABVOL0i: Difference between the stock trading volume on day 0 and its average volume from days -250 to -1.

1. **ARit**

In [ ]:
AR_df_pre = pd.read_excel(f"{path}/raw_data/pre_covid/ARs_df.xlsx")
AR_df_post = pd.read_excel(f"{path}/raw_data/post_covid/ARs_regression.xlsx")

2. **MR_dum**

In [ ]:
proxy_a_df_pre = pd.read_excel(f"{path}/raw_data/pre_covid/proxy_a.xlsx")
mr_dum_pre = proxy_a_df_pre[['Date','Market_Return_Increase']]
proxy_a_df_post = pd.read_excel(f"{path}/raw_data/post_covid/proxy_a.xlsx")
mr_dum_post = proxy_a_df_post[['Date','Market_Return_Increase']]

3. **MCap**

Since we didn't fetch the market capitalization earlier, we need to do it now:

In [ ]:
tickers_total= pd.read_excel(f"{path}/raw_data/main/Weightings_20240216_OMXSPI.xlsx",header=0)
tickers_list=tickers_total['Security-Symbol'].to_list()
price_2013_pre=pd.read_excel(f"{path}/raw_data/pre_covid/price_2013.xlsx",header=0)
# tickers = yf.Tickers(tickers_list) #, start="2014-03-06", end='2020-03-14')
for t in ['8TRA.ST','AAK.ST']: #tickers_list:
    ticker= yf.Ticker(f'{t}')
    quarterly_income_stmt = ticker.quarterly_income_stmt
    print(quarterly_income_stmt.columns)
    # all_dates = ticker.quarterly_income_stmt.columns.to_list
    # print(all_dates)
    # total_shares = ticker.quarterly_income_stmt[date]['Basic Average Shares']

4. **Beta**

In [ ]:
df_price_2013_pre=pd.read_excel(f"{path}/raw_data/pre_covid/price_2013.xlsx",header=0)
df_price_2013_pre['market_returns'] =df_price_2013_pre['^OMXSPI'].pct_change()
window_size=250
beta_pre={'Date':df_price_2013_pre['Date']}
for column in df_price_2013_pre.columns[1:-2]:  # Exclude the 'Date' and '^OMXSPI' column

    stock_returns = df_price_2013_pre[column].pct_change()
    result_df = pd.DataFrame({
    'Date': df_price_2013_pre['Date'],
    'Stock_Returns': stock_returns,
    'Market_Returns': df_price_2013_pre['market_returns']})

    result_df['beta'] = result_df['Stock_Returns'].rolling(window=window_size).cov(result_df['Market_Returns']).div(result_df['Market_Returns'].rolling(window=window_size).var())
    result_df['normalized_beta'] = (result_df['beta'] - result_df['beta'].mean())/result_df['beta'].std()

    beta_pre[f'beta_{column}']= result_df['normalized_beta']
beta_pre=pd.DataFrame.from_dict(beta_pre)
beta_pre

In [ ]:
df_price_2013_post=pd.read_excel(f"{path}/raw_data/main/price_2013.xlsx",header=0)
df_price_2013_post['market_returns'] =df_price_2013_post['^OMXSPI'].pct_change()
window_size=250
beta_post={'Date':df_price_2013_post['Date']}
for column in df_price_2013_post.columns[1:-2]:  # Exclude the 'Date' and '^OMXSPI' column

    stock_returns = df_price_2013_post[column].pct_change()
    result_df = pd.DataFrame({
    'Date': df_price_2013_post['Date'],
    'Stock_Returns': stock_returns,
    'Market_Returns': df_price_2013_post['market_returns']})

    result_df['beta'] = result_df['Stock_Returns'].rolling(window=window_size).cov(result_df['Market_Returns']).div(result_df['Market_Returns'].rolling(window=window_size).var())
    result_df['normalized_beta'] = (result_df['beta'] - result_df['beta'].mean())/result_df['beta'].std()
    beta_post[f'beta_{column}']= result_df['normalized_beta']
beta_post=pd.DataFrame.from_dict(beta_post)
beta_post=beta_post[beta_post['Date']>='2020-03-14']
beta_post

5. **SR_vola**

In [ ]:
df_price_2013_pre=pd.read_excel(f"{path}/raw_data/pre_covid/price_2013.xlsx",header=0)
df_sr_vola_pre_dict={'Date':df_price_2013_pre['Date']}
window_size=250
for column in df_price_2013_pre.columns[1:-1]:  # Exclude the 'Date' and '^OMXSPI' column

    stock_returns = df_price_2013_pre[column].pct_change()
    result_df = pd.DataFrame({
    'Date': df_price_2013_pre['Date'],
    'Stock_Returns': stock_returns})

    result_df['sr_vola']= stock_returns.rolling(window=window_size).std()
    result_df['normalized_sr_vola']= (result_df['sr_vola']-result_df['sr_vola'].mean())/result_df['sr_vola'].std()


    df_sr_vola_pre_dict[f'sr_vola_{column}']=result_df['normalized_sr_vola']
df_sr_vola_pre=pd.DataFrame.from_dict(df_sr_vola_pre_dict)
df_sr_vola_pre.head(400)

In [ ]:
df_price_2013_post=pd.read_excel(f"{path}/raw_data/main/price_2013.xlsx",header=0)
df_sr_vola_post_dict={'Date':df_price_2013_post['Date']}
window_size=250
for column in df_price_2013_post.columns[1:-1]:  # Exclude the 'Date' and '^OMXSPI' column

    stock_returns = df_price_2013_post[column].pct_change()
    result_df = pd.DataFrame({
    'Date': df_price_2013_post['Date'],
    'Stock_Returns': stock_returns})

    result_df['sr_vola']= stock_returns.rolling(window=window_size).std()
    result_df['normalized_sr_vola']= (result_df['sr_vola']-result_df['sr_vola'].mean())/result_df['sr_vola'].std()

    df_sr_vola_post_dict[f'sr_vola_{column}']=result_df['normalized_sr_vola']
df_sr_vola_post=pd.DataFrame.from_dict(df_sr_vola_post_dict)
df_sr_vola_post=df_sr_vola_post[df_sr_vola_post['Date']>='2020-03-14']
df_sr_vola_post.head(400)

6.  **|SRo|i**

In [ ]:
df_price_2013_pre=pd.read_excel(f"{path}/raw_data/pre_covid/price_2013.xlsx",header=0)
df_abs_sr_pre_dict={'Date':df_price_2013_pre['Date']}
for column in df_price_2013_pre.columns[1:-1]:  # Exclude the 'Date' and '^OMXSPI' column

    stock_returns = df_price_2013_pre[column].pct_change()
    df_abs_sr_pre_dict[f'abs_sr_{column}'] = abs(stock_returns)
df_abs_sr_pre = pd.DataFrame.from_dict(df_abs_sr_pre_dict)
df_abs_sr_pre.head()

In [ ]:
df_price_2013_post=pd.read_excel(f"{path}/raw_data/main/price_2013.xlsx",header=0)
df_abs_sr_post_dict={'Date':df_price_2013_post['Date']}
for column in df_price_2013_post.columns[1:-1]:  # Exclude the 'Date' and '^OMXSPI' column

    stock_returns = df_price_2013_post[column].pct_change()
    df_abs_sr_post_dict[f'abs_sr_{column}'] = abs(stock_returns)
df_abs_sr_post = pd.DataFrame.from_dict(df_abs_sr_post_dict)
df_abs_sr_post = df_abs_sr_post[df_abs_sr_post['Date']>='2020-03-14']
df_abs_sr_post.head()

7. **ABVOL**

In [ ]:
volume_pre = pd.read_excel(f'{path}/raw_data/pre_covid/volume.xlsx')
window_size=250
df_ab_vol_pre_dict = {'Date':volume_pre['Date']}

for column in volume_pre.columns[1:]:
    rolling_mean = volume_pre[column].rolling(window=window_size, min_periods=1).mean()
    rolling_std = volume_pre[column].rolling(window=window_size, min_periods=1).std()
    df_ab_vol_pre_dict[f'ab_vol_{column}']=(volume_pre[column] - rolling_mean) / rolling_std
df_ab_vol_pre = pd.DataFrame.from_dict(df_ab_vol_pre_dict)
df_ab_vol_pre.head(400)

In [ ]:
volume_post = pd.read_excel(f'{path}/raw_data/main/volume.xlsx')
window_size=250
df_ab_vol_post_dict = {'Date':volume_post['Date']}

for column in volume_post.columns[1:]:
    rolling_mean = volume_post[column].rolling(window=window_size, min_periods=1).mean()
    rolling_std = volume_post[column].rolling(window=window_size, min_periods=1).std()
    df_ab_vol_post_dict[f'ab_vol_{column}']=(volume_post[column] - rolling_mean) / rolling_std
df_ab_vol_post = pd.DataFrame.from_dict(df_ab_vol_post_dict)
df_ab_vol_post = df_ab_vol_post[df_ab_vol_post['Date']>='2020-03-14']
df_ab_vol_post.head(400)

Now it's time to merge all the variables together.

Additionaly, once at a time, we will join one proxy, in order to be able to stand out the events.


In [ ]:
total_variables_pre = pd.merge(AR_df_pre,mr_dum_pre,on='Date',how='left')
total_variables_pre = pd.merge(total_variables_pre,beta_pre,on='Date',how='left')
total_variables_pre = pd.merge(total_variables_pre,df_sr_vola_pre,on='Date',how='left')
total_variables_pre = pd.merge(total_variables_pre,df_abs_sr_pre,on='Date',how='left')
total_variables_pre = pd.merge(total_variables_pre,df_ab_vol_pre,on='Date',how='left')
total_variables_pre.drop(columns=['Unnamed: 0','Swedish Treasury Bills (SE TB 1 Month)','market_returns'],inplace=True)
total_variables_pre

In [ ]:
total_variables_post = pd.merge(AR_df_post,mr_dum_post,on='Date',how='left')
total_variables_post = pd.merge(total_variables_post,beta_post,on='Date',how='left')
total_variables_post = pd.merge(total_variables_post,df_sr_vola_post,on='Date',how='left')
total_variables_post = pd.merge(total_variables_post,df_abs_sr_post,on='Date',how='left')
total_variables_post = pd.merge(total_variables_post,df_ab_vol_post,on='Date',how='left')
total_variables_post.drop(columns=['Swedish Treasury Bills (SE TB 1 Month)','market_returns'],inplace=True)
total_variables_post

# Regression Model

In [ ]:
from linearmodels.panel import PanelOLS

## Pre-Covid

### Proxy A

#### Small Threshold

In [ ]:
total_df_a_pre = pd.merge(total_variables_pre,proxy_a_df_pre, on='Date', how='left')
